# Explore OMA database

They have pairwise orthologs identified for 20 mil proteins, 2.2k genomes.
They track taxon IDs. Many proteins link back to uniprot.

4 billion ortholog pairs identified, how many of them correspond to meso-thermo pairs?

join their data with OGT data and find out...

In [25]:
import duckdb as ddb
conn = ddb.connect('./oma.db', read_only=False)

## Ortho pairs

In [28]:
conn.execute("""
    CREATE OR REPLACE TABLE pairs AS SELECT * FROM
        read_csv_auto(
            './data/pairs/x00000000',
            header=False,
            sep='\t',
            nullstr='n/a',
            columns={'id1':STRING, 'id2':STRING, 'type':STRING, 'group':INTEGER},
            parallel=True)
    """)
conn.commit()

In [29]:
conn.execute("SELECT * FROM pairs").df()

,id1,id2,type,group
0,HEIAB00001,LOKAC00321,1:1,1159110.0
1,HEIAB00001,ACIB401296,1:1,1159110.0
2,HEIAB00001,ACIS001187,1:1,1159110.0
3,HEIAB00001,METII01272,1:1,1159110.0
4,HEIAB00001,THOAR00839,1:1,1159110.0
...,...,...,...,...
99995,HEIAB00174,KLEAK00333,1:1,1050333.0
99996,HEIAB00174,KLEOK00140,1:1,1050333.0
99997,HEIAB00174,KLEP301040,1:1,1050333.0
99998,HEIAB00174,KLEP702989,1:1,1050333.0


### Send it on 4 billion pairs

In [39]:
conn.execute("""
    CREATE OR REPLACE TABLE pairs AS SELECT * FROM
        read_csv_auto(
            './data/pairs/x*',
            header=False,
            sep='\t',
            nullstr='n/a',
            columns={'id1':STRING, 'id2':STRING, 'type':STRING, 'group':INTEGER},
            parallel=True)
    """)
conn.commit()

FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

IOException: IO Error: Could not write file "/Users/ek/Documents/Research/Projects/thermoshift/learn2thermDB/testing/oma/./oma.db.tmp/duckdb_temp_storage-123.tmp": No space left on device

## Proteins

In [34]:
conn.execute("""CREATE OR REPLACE TABLE proteins AS SELECT * FROM
    read_csv_auto('./data/uniprot_map/oma-uniprot.txt', sep='\t', header=False, nullstr='n/a',
        columns={'oma_id':STRING, 'uniprot_id':STRING}, skip=4)""")

FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

In [35]:
conn.execute("SELECT * FROM proteins LIMIT 10").df()

,oma_id,uniprot_id
0,HEIAB00001,A0A2U3CKJ0
1,HEIAB00002,A0A2U3CKK2
2,HEIAB00003,A0A2U3CKK9
3,HEIAB00004,A0A2U3CKI6
4,HEIAB00005,A0A2U3CKJ5
5,HEIAB00006,A0A2U3CKL8
6,HEIAB00007,A0A2U3CKJ8
7,HEIAB00008,A0A2U3CKJ1
8,HEIAB00009,A0A2U3CKL6
9,HEIAB00010,A0A2U3CKI3


In [36]:
conn.execute("SELECT COUNT(DISTINCT(uniprot_id)) FROM proteins").df()

,count(DISTINCT uniprot_id)
0,17474460


In [37]:
conn.execute("SELECT COUNT(*) FROM proteins").df()

,count_star()
0,18807037


In [38]:
conn.execute("SELECT COUNT(*) FROM proteins WHERE uniprot_id IS NULL").df()

,count_star()
0,0


Seems like there is a duplicate mapping some places